##### Tahap 1: Memuat Data
Pertama, kita perlu memuat data dari file Excel yang telah Anda sediakan. Kita akan menggunakan pustaka pandas untuk ini.

In [30]:
import pandas as pd

# Memuat data dari file Excel
kepadatan_data = pd.read_excel('Kepadatan_Penduduk_menurut_Provinsi.xlsx')
jumlah_data = pd.read_excel('Jumlah_Penduduk.xlsx')
covid_data = pd.read_excel('Statistik_Harian_per_Provinsi_COVID19_Indonesia_Rev.xlsx')

##### Tahap 2: Memeriksa dan Menyesuaikan Nama Kolom
Sebelum menggabungkan data, kita perlu memeriksa nama kolom di setiap DataFrame untuk memastikan bahwa kita dapat menggabungkannya dengan benar.

In [37]:
# Cek nama kolom di setiap DataFrame
print("Kepadatan Data Columns:", kepadatan_data.columns.tolist())
print("Jumlah Data Columns:", jumlah_data.columns.tolist())
print("COVID Data Columns:", covid_data.columns.tolist())

Kepadatan Data Columns: ['Unnamed: 0', 'Province', 'Kepadatan Penduduk menurut Provinsi (jiwa/km2)', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11']
Jumlah Data Columns: ['Province', 'Jumlah Penduduk Hasil Proyeksi Menurut Provinsi dan Jenis Kelamin (Ribu Jiwa)', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9']
COVID Data Columns: ['Object_ID', 'Province', 'Tanggal', 'Kasus_Terkonfirmasi_Akumulatif', 'Penambahan_Harian_Kasus_Terkonf', 'Kasus_Sembuh_Akumulatif', 'Penambahan_Harian_Kasus_Sembuh', 'Kasus_Meninggal_Akumulatif', 'Penambahan_Harian_Kasus_Meningg', 'Kasus_Aktif_Akumulatif', 'CFR_Harian', 'RI_Harian', 'FID', 'ObjectId']


Jika nama kolom tidak konsisten (misalnya, ada spasi atau perbedaan huruf besar/kecil), kita perlu menyesuaikannya. Misalnya, jika kolom provinsi di semua DataFrame tidak konsisten, kita bisa menggantinya.

In [32]:
# Mengganti nama kolom jika diperlukan
kepadatan_data.rename(columns={'Provinsi': 'Province'}, inplace=True)
jumlah_data.rename(columns={'Provinsi': 'Province'}, inplace=True)
covid_data.rename(columns={'Provinsi': 'Province'}, inplace=True)

##### Tahap 3: Menggabungkan Data
Setelah memastikan nama kolom konsisten, kita akan menggabungkan ketiga DataFrame berdasarkan kolom 'Province'.

In [36]:
# Menggabungkan dataset berdasarkan nama provinsi
merged_data = pd.merge(kepadatan_data, jumlah_data, on='Province')
merged_data = pd.merge(merged_data, covid_data, on='Province')

# Cek nama kolom setelah penggabungan
print("Merged Data Columns:", merged_data.columns.tolist())

Merged Data Columns: ['Unnamed: 0', 'Province', 'Kepadatan Penduduk menurut Provinsi (jiwa/km2)', 'Unnamed: 3_x', 'Unnamed: 4_x', 'Unnamed: 5_x', 'Unnamed: 6_x', 'Unnamed: 7_x', 'Unnamed: 8_x', 'Unnamed: 9_x', 'Unnamed: 10', 'Unnamed: 11', 'Jumlah Penduduk Hasil Proyeksi Menurut Provinsi dan Jenis Kelamin (Ribu Jiwa)', 'Unnamed: 2', 'Unnamed: 3_y', 'Unnamed: 4_y', 'Unnamed: 5_y', 'Unnamed: 6_y', 'Unnamed: 7_y', 'Unnamed: 8_y', 'Unnamed: 9_y', 'Object_ID', 'Tanggal', 'Kasus_Terkonfirmasi_Akumulatif', 'Penambahan_Harian_Kasus_Terkonf', 'Kasus_Sembuh_Akumulatif', 'Penambahan_Harian_Kasus_Sembuh', 'Kasus_Meninggal_Akumulatif', 'Penambahan_Harian_Kasus_Meningg', 'Kasus_Aktif_Akumulatif', 'CFR_Harian', 'RI_Harian', 'FID', 'ObjectId']


##### Tahap 4: Memilih Atribut untuk Clustering
Kita perlu memilih 8 atribut yang relevan untuk analisis clustering. Atribut ini bisa bervariasi tergantung pada data yang tersedia, tetapi kita akan memilih beberapa yang umum digunakan.

In [45]:
# Memilih 8 atribut yang relevan
features = merged_data[['Population Density', 'Total Population', 'COVID Cases', 'GDP', 'Urbanization Rate', 'Health Index', 'Education Level', 'Employment Rate']]

KeyError: "None of [Index(['Population Density', 'Total Population', 'COVID Cases', 'GDP',\n       'Urbanization Rate', 'Health Index', 'Education Level',\n       'Employment Rate'],\n      dtype='object')] are in the [columns]"

##### Tahap 5: Normalisasi Data
Sebelum melakukan clustering, kita perlu menormalkan data agar semua fitur memiliki skala yang sama. Kita akan menggunakan StandardScaler dari pustaka sklearn.

In [39]:
from sklearn.preprocessing import StandardScaler

# Normalisasi data
scaler = StandardScaler()
normalized_data = scaler.fit_transform(features)

NameError: name 'features' is not defined

##### Tahap 6: Clustering
Kita akan menggunakan algoritma K-Means untuk melakukan clustering dengan jumlah kelompok yang berbeda (3, 4, 5, 6, dan 7). Kita juga akan memvisualisasikan hasil clustering.

In [40]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Menentukan jumlah cluster
cluster_numbers = [3, 4, 5, 6, 7]
cluster_labels = {}

for n_clusters in cluster_numbers:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(normalized_data)
    cluster_labels[n_clusters] = labels

    # Visualisasi hasil clustering
    plt.figure(figsize=(8, 6))
    plt.scatter(normalized_data[:, 0], normalized_data[:, 1], c=labels, cmap='viridis')
    plt.title(f'K-Means Clustering dengan {n_clusters} Kelompok')
    plt.xlabel('Fitur 1')
    plt.ylabel('Fitur 2')
    plt.show()

NameError: name 'normalized_data' is not defined

##### Tahap 7: Menetapkan Zona
Setelah melakukan clustering, kita akan menetapkan zona (Hitam, Merah, Kuning, Hijau) berdasarkan hasil clustering. Kita akan menggunakan pemetaan sederhana untuk ini.

In [41]:
# Menetapkan zona berdasarkan label cluster
zone_mapping = {
    0: 'Hijau',
    1: 'Kuning',
    2: 'Merah',
    3: 'Hitam',
    # Tambahkan lebih banyak pemetaan jika diperlukan
}

# Menambahkan kolom zona ke data yang digabungkan
for n_clusters, labels in cluster_labels.items():
    merged_data[f'Zone_{n_clusters}'] = [zone_mapping[label] for label in labels]

##### Tahap 8: Menyimpan Hasil
Terakhir, kita akan menyimpan dataset akhir dengan zona yang ditetapkan ke dalam file Excel.